In [1]:
import sys, os, time
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%matplotlib tk
import matplotlib.pyplot as plt

import tnt_util as util
from tnt_util import adict, idict, xset, collate, load, render_dict, save, Logger, seq_iterate
from tnt_setup import init_gamestate, setup_phase
from tnt_cards import load_card_decks, draw_cards
from collections import namedtuple
import random
from itertools import chain, product
from tnt_units import load_unit_rules


import tnt_setup as setup

from flask_app import *

In [2]:
ping()

'Backend active: use "init" to init game'

In [3]:
player = 'Axis'

In [4]:
out = format_msg_to_python(init_game(debug=False, player=player))

Beginning phase: Setup


In [5]:
path = 'saves/setup_complete.json'
load(path)

Game loaded


In [6]:
player = 'West'

In [7]:
player = 'USSR'

In [8]:
player

'USSR'

In [9]:
out = format_msg_to_python(get_status(player))

In [15]:
if 'log' in out:
    print(out.log)

West spends 1 production on building a new cadre in London (5 production remaining)
West spends 1 production on building a new cadre in Marseille (4 production remaining)
West spends 1 production on upgrading a cadre in London (3 production remaining)
West spends 1 production on upgrading a cadre in London (2 production remaining)
West spends 1 production on upgrading a cadre in Lorraine (1 production remaining)
West spends 1 production on building a new cadre in Glasgow (0 production remaining)
West draws 0 action cards (now holding 8 cards)
West draws 0 investment cards (now holding 8 cards)
West is done with production



In [14]:
render_dict(out)

In [16]:
for player in ['Axis', 'West', 'USSR']:
    out = format_msg_to_python(get_status(player))
    for _ in range(25):
        if 'error' in out:
            print(out.error)
            break
        elif 'actions' not in out:
            #render_dict(out)
            print('Player {} is waiting'.format(player))
            break
        action = random.choice(list(util.decode_actions(out.actions)))
        print('--- Playing',action)
        out = format_msg_to_python(take_action(player, action))
        if 'log' in out and len(out.log)>0:
            print(out.log)

--- Playing ('Germany', 'Ruhr', 'Carrier')
Axis spends 1 production on building a new cadre in Ruhr (10 production remaining)

--- Playing (126,)
Axis spends 1 production on upgrading a cadre in Munich (9 production remaining)

--- Playing ('Italy', 'Sardinia', 'Fleet')
Axis spends 1 production on building a new cadre in Sardinia (8 production remaining)

--- Playing ('Germany', 'Munich', 'Infantry')
Axis spends 1 production on building a new cadre in Munich (7 production remaining)

--- Playing (130,)
Axis spends 1 production on upgrading a cadre in Rome (6 production remaining)

--- Playing (119,)
Axis spends 1 production on upgrading a cadre in Ruhr (5 production remaining)

--- Playing (118,)
Axis spends 1 production on upgrading a cadre in Tripoli (4 production remaining)

--- Playing ('Italy', 'Sicily', 'AirForce')
Axis spends 1 production on building a new cadre in Sicily (3 production remaining)

--- Playing ('Italy', 'Taranto', 'Infantry')
Axis spends 1 production on building 

In [27]:
actions = list(util.decode_actions(out.actions))
action = random.choice(actions)
out = format_msg_to_python(take_action(player, action))
if 'error' in out:
    print(out.error)
if 'log' in out:
    print(out.log)
print('--- Playing from {}: {}'.format(len(actions),action))

KeyError: 'actions'

In [13]:

action

In [15]:
#out = format_msg_to_python(take_action(player, ('Britain', 'Paris', 'Infantry')))

--- Playing ('Italy', 'Rome', 'Submarine')
--- Playing ('Germany', 'Konigsberg', 'Infantry')
--- Playing ('Germany', 'Konigsberg', 'Infantry')
--- Playing ('Germany', 'Berlin', 'Submarine')
--- Playing ('Germany', 'Berlin', 'Infantry')
--- Playing ('Italy', 'Tripoli', 'Tank')
--- Playing ('Italy', 'Milan', 'AirForce')
--- Playing ('Germany', 'Ruhr', 'Carrier')
--- Playing ('Italy', 'Milan', 'Fortress')
--- Playing ('Italy', 'Rome', 'Carrier')
--- Playing ('Italy', 'Tripoli', 'Fleet')
--- Playing ('Germany', 'Berlin', 'Submarine')
--- Playing ('Germany', 'Munich', 'AirForce')
--- Playing ('Germany', 'Ruhr', 'Carrier')
--- Playing ('Germany', 'Munich', 'Infantry')
--- Playing ('Italy', 'Rome', 'Fleet')
--- Playing ('Italy', 'Rome', 'Tank')
--- Playing ('Germany', 'Berlin', 'AirForce')
--- Playing ('Germany', 'Ruhr', 'Fortress')
--- Playing ('Germany', 'Berlin', 'Tank')
--- Playing ('Germany', 'Ruhr', 'Tank')
--- Playing ('Germany', 'Berlin', 'Submarine')
Axis draws 14 action cards (now h

In [16]:
G = get_G()
render_dict(G)

In [1]:
W = get_waiting_actions()
render_dict(W)

NameError: name 'get_waiting_actions' is not defined

In [11]:
for o in seq_iterate(G.players, [None, 'homeland', None], end=True):
    print(o)

('Axis', 'Germany', {'Konigsberg', 'Ruhr', 'Berlin', 'Munich'})
('Axis', 'Italy', {'Taranto', 'Sicily', 'Venice', 'Sardinia', 'Milan', 'Rome'})
('USSR', 'USSR', {'Stalingrad', 'Kazakhstan', 'Gorky', 'Belorussia', 'Voronezh', 'Archangel', 'Kuban', 'Odessa', 'Kharkov', 'Bryansk', 'Murmansk', 'Penza', 'Georgia', 'Moscow', 'Perm', 'Leningrad', 'Western_Siberia', 'Baku', 'Urals', 'Sevastopol', 'Kiev', 'Grozny', 'Turkmenistan', 'Vologda', 'Ufa'})
('West', 'Britain', {'London', 'Glasgow'})
('West', 'France', {'Paris', 'Lorraine', 'Marseille', 'Gascony'})


In [7]:
# path = save('setup_complete.json')
# path

Game saved


'saves/setup_complete.json'

In [7]:
path = 'saves/init_complete.json'
load(path)

Game loaded


In [2]:
a = adict()
load_unit_rules(a)
render_dict(a)